In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

### Read data
Read the data using `read_csv()`

In [ ]:
df = pd.read_csv('telecom_churn.csv')

Take a look at the first 5 lines using `head` method:

In [ ]:
df.head()

Dimensionality

In [ ]:
df.shape

Column names

In [ ]:
df.columns

In [ ]:
df.info()

`bool`, `int64`, `float64` and `object` are the data types of our features. We see that one feature is logical (`bool`), 3 features are of type `object`, and 16 features are numeric. With this same method, we can easily see if there are any missing values. Here, there are none because each column contains 3333 observations, the same number of rows we saw before with `shape`.

We can **change the column type** with the `astype` method. Let’s apply this method to the `Churn` feature to convert it into `int64`:



In [ ]:
df['Churn'] = df['Churn'].astype('int64')


The `describe` method shows basic statistical characteristics of each numerical feature (`int64` and `float64` types): number of non-missing values, mean, standard deviation, range, median, 0.25 and 0.75 quartiles.

In [ ]:
df.describe()

In order to see statistics on non-numerical features, one has to explicitly indicate data types of interest in the `include` parameter.

In [ ]:
df.describe(include=['object', 'bool'])

For categorical (type `object`) and boolean (type `bool`) features we can use the `value_counts` method. Let's take a look at the distribution of `Churn`:

In [ ]:
df['Churn'].value_counts()

2850 users out of 3333 are *loyal*; their `Churn` value is 0. To calculate fractions, pass `normalize=True` to the `value_counts` function.


In [ ]:
df['Churn'].value_counts(normalize=True)

---


### Sorting

A `DataFrame` can be sorted by the value of one of the variables (i.e columns). For example, we can sort by *Total day charge* (use `ascending=False` to sort in descending order):



In [ ]:
df.sort_values(by='Total day charge', ascending=False).head()

In [ ]:
df.sort_values(by=['Churn', 'Total day charge'], ascending=[True, False]).head()

---


### Indexing and retrieving data

A `DataFrame` can be indexed in a few different ways. 

To get a single column, you can use a `DataFrame['Name']` construction. Let's use this to answer a question about that column alone: **what is the proportion of churned users in our dataframe?**




In [ ]:
df['Churn'].mean()

**Boolean indexing** with one column is also very convenient. The syntax is `df[P(df['Name'])]`, where `P` is some logical condition that is checked for each element of the `Name` column. The result of such indexing is the `DataFrame` consisting only of rows that satisfy the `P` condition on the `Name` column. 

Let's use it to answer the question:

**What are average values of numerical features for churned users?**

In [ ]:
df[df['Churn'] == 1].mean()

**How much time (on average) do churned users spend on the phone during daytime?**

In [ ]:
df[df['Churn'] == 1]['Total day minutes'].mean()


**What is the maximum length of international calls among loyal users (`Churn == 0`) who do not have an international plan?**




In [ ]:
df[(df['Churn'] == 0) & (df['International plan'] == 'No')]['Total intl minutes'].max()


DataFrames can be indexed by column name (label) or row name (index) or by the serial number of a row. The `loc` method is used for **indexing by name**, while `iloc()` is used for **indexing by number**.

In the first case below, we say *"give us the values of the rows with index from 0 to 5 (inclusive) and columns labeled from State to Area code (inclusive)"*. In the second case, we say *"give us the values of the first five rows in the first three columns"* (as in a typical Python slice: the maximal value is not included).



In [ ]:
df.loc[0:5, 'State':'Area code']

In [ ]:
df.iloc[0:5, 0:3]

If we need the first or the last line of the data frame, we can use the `df[:1]` or `df[-1:]` construction:

In [ ]:
df[-1:]

---


### Applying Functions to Cells, Columns and Rows

**To apply functions to each column, use `apply()`:**



In [ ]:
df.apply(np.max) 

The `apply` method can also be used to apply a function to each row. To do this, specify `axis=1`. Lambda functions are very convenient in such scenarios. For example, if we need to select all states starting with 'W', we can do it like this:

In [ ]:
df[df['State'].apply(lambda state: state[0] == 'W')].head()

The `map` method can be used to **replace values in a column** by passing a dictionary of the form `{old_value: new_value}` as its argument:

In [ ]:
d = {'No' : False, 'Yes' : True}
df['International plan'] = df['International plan'].map(d)
df.head()

The same thing can be done with the `replace` method:

In [ ]:
df = df.replace({'Voice mail plan': d})
df.head()

When to use map, applymap and apply
https://stackoverflow.com/questions/19798153/difference-between-map-applymap-and-apply-methods-in-pandas

---


### Grouping

In general, grouping data in Pandas works as follows:




```python
df.groupby(by=grouping_columns)[columns_to_show].function()
```


1. First, the `groupby` method divides the `grouping_columns` by their values. They become a new index in the resulting dataframe.
2. Then, columns of interest are selected (`columns_to_show`). If `columns_to_show` is not included, all non groupby clauses will be included.
3. Finally, one or several functions are applied to the obtained groups per selected columns.

Here is an example where we group the data according to the values of the `Churn` variable and display statistics of three columns in each group:

In [ ]:

columns_to_show = ['Total day minutes', 
                   'Total eve minutes', 
                   'Total night minutes']

df.groupby(['Churn'])[columns_to_show].describe(percentiles=[])

Let’s do the same thing, but slightly differently by passing a list of functions to `agg()`:

In [ ]:
columns_to_show = ['Total day minutes', 
                   'Total eve minutes', 
                   'Total night minutes']

df.groupby(['Churn'])[columns_to_show].agg([np.mean, np.std, np.min, np.max])

---


### Summary tables

Suppose we want to see how the observations in our dataset are distributed in the context of two variables - `Churn` and `International plan`. To do so, we can build a **contingency table** using the `crosstab` method:




In [ ]:
pd.crosstab(df['Churn'], df['International plan'])

In [ ]:
pd.crosstab(df['Churn'], df['Voice mail plan'], normalize=True)

We can see that most of the users are loyal and do not use additional services (International Plan/Voice mail).

This will resemble **pivot tables** to those familiar with Excel. And, of course, pivot tables are implemented in Pandas: the `pivot_table` method takes the following parameters:

* `values` – a list of variables to calculate statistics for,
* `index` – a list of variables to group data by,
* `aggfunc` – what statistics we need to calculate for groups, ex. sum, mean, maximum, minimum or something else.

Let’s take a look at the average number of day, evening, and night calls by area code:

In [ ]:
df.pivot_table(['Total day calls', 'Total eve calls', 'Total night calls'],
               ['State'], aggfunc=['mean','max'])

In [ ]:
df.pivot_table(['Total day calls', 'Total eve calls', 'Total night calls'],
               ['Area code'], aggfunc='mean')

In [ ]:
df.pivot_table(['Total day calls', 'Total eve calls', 'Total night calls'],
               ['Area code'], aggfunc=['max', 'mean', 'median'])

Pivot table expects a dataframe whereas cross_tab can accept array-like objects

---


### DataFrame transformations

Like many other things in Pandas, adding columns to a DataFrame is doable in many ways.

For example, if we want to calculate the total number of calls for all users, let’s create the `total_calls` Series and paste it into the DataFrame:




In [ ]:
total_calls = df['Total day calls'] + df['Total eve calls'] + \
              df['Total night calls'] + df['Total intl calls']
df.insert(loc=len(df.columns), column='Total calls', value=total_calls) 
# loc parameter is the number of columns after which to insert the Series object
# we set it to len(df.columns) to paste it at the very end of the dataframe
df.head()

It is possible to add a column more easily without creating an intermediate Series instance:

In [ ]:
df['Total charge'] = df['Total day charge'] + df['Total eve charge'] + \
                     df['Total night charge'] + df['Total intl charge']
df.head()

To delete columns or rows, use the `drop` method, passing the required indexes and the `axis` parameter (`1` if you delete columns, and nothing or `0` if you delete rows). The `inplace` argument tells whether to change the original DataFrame. With `inplace=False`, the `drop` method doesn't change the existing DataFrame and returns a new one with dropped rows or columns. With `inplace=True`, it alters the DataFrame.

In [ ]:
df.drop(['Total charge'], axis= 1, inplace=True)

#or 

df = df.drop(['Total charge'], axis= 1)

In [ ]:
# get rid of just created columns
df.drop(['Total charge', 'Total calls'], axis=1, inplace=True) 
# and here’s how you can delete rows
df.drop([1, 2]).head() 

---

## Data visualisation with Pandas

In [ ]:
df.groupby('State')['Churn'].sum().plot(figsize=(14, 8), kind='bar')

In [ ]:
df.groupby('State')['Churn'].sum().plot(figsize=(14, 12), kind='barh')

In [ ]:
df.groupby('State')['Churn'].sum().sort_values().plot(figsize=(14, 12), kind='barh')

In [ ]:
df.groupby('State')['Churn'].sum().sort_values()[-15:].plot(figsize=(14, 6), kind='barh')

### Histogram

In [ ]:
df[['Total day calls']].plot(kind='hist')#,bins=[0,20,40,60,80,100],rwidth=0.8)


In [ ]:
df[['Total day calls']].plot(kind='hist',bins=range(0,200,10),rwidth=0.8)

### Scatter plot

In [ ]:
df.plot(kind='scatter',x='Total calls',y='Total night calls',color='red')


In [ ]:
df.plot(kind='scatter',x='Total day calls',y='Total night calls',color='red')

